# How to Generate ERV index in GTF format from GFF3 file?

hERV downloaded from https://herv.img.cas.cz/downloads is gff3; Mouse Erv was generated by siyi using repeatmasker and compared with [gEVE](http://geve.med.u-tokai.ac.jp/about/) to validate.
Well, it's quite simple, really. Also see `gen_gtf_from_gff3.sh`. The OG file was in this format:

In [ ]:
chr1    .       BED_feature             34597   34659   500     -       .       ID=BED_feature1;Name=12:ERV:MLT1L
chr1    .       BED_thick_feature       34597   34659   500     -       .       Parent=BED_feature1;Name=12:ERV:MLT1L
chr1    .       BED_block               34597   34659   500     -       .       Parent=BED_feature1;Name=12:ERV:MLT1L
###
chr1    .       BED_feature             34566   34972   500     -       .       ID=BED_feature2;Name=14:ERV:MLT1J2
chr1    .       BED_thick_feature       34566   34972   500     -       .       Parent=BED_feature2;Name=14:ERV:MLT1J2
chr1    .       BED_block               34566   34972   500     -       .       Parent=BED_feature2;Name=14:ERV:MLT1J2
###
chr1    .       BED_feature             40850   41007   500     -       .       ID=BED_feature3;Name=18:ERV:LTR16C
chr1    .       BED_thick_feature       40850   41007   500     -       .       Parent=BED_feature3;Name=18:ERV:LTR16C
chr1    .       BED_block               40850   41007   500     -       .       Parent=BED_feature3;Name=18:ERV:LTR16C
###
chr1    .       BED_feature             41804   42717   500     -       .       ID=BED_feature4;Name=19:ERV:ERV3-16A3_I
chr1    .       BED_thick_feature       41804   42717   500     -       .       Parent=BED_feature4;Name=19:ERV:ERV3-16A3_I
chr1    .       BED_block               41804   42717   500     -       .       Parent=BED_feature4;Name=19:ERV:ERV3-16A3_I

First, we select the lines with only `BED_feature` labels. The `-F"\t"` indicates to `awk` that we're dealing with .tsv.

In [ ]:
awk -F"\t" '$3==BED_feature' erv.gff3 # I forgot this command :(

This results in this format:

In [ ]:
chr1    .       BED_feature     34597   34659   500     -       .       ID=BED_feature1;Name=12:ERV:MLT1L
chr1    .       BED_feature     34566   34972   500     -       .       ID=BED_feature2;Name=14:ERV:MLT1J2
chr1    .       BED_feature     40850   41007   500     -       .       ID=BED_feature3;Name=18:ERV:LTR16C
chr1    .       BED_feature     41804   42717   500     -       .       ID=BED_feature4;Name=19:ERV:ERV3-16A3_I
chr1    .       BED_feature     46766   46842   500     -       .       ID=BED_feature5;Name=20:ERV:LTR12B
chr1    .       BED_feature     70764   70893   500     -       .       ID=BED_feature6;Name=29:ERV:LTR89
chr1    .       BED_feature     91610   91678   500     -       .       ID=BED_feature7;Name=36:ERV:MER4E1
chr1    .       BED_feature     92187   92477   500     -       .       ID=BED_feature8;Name=37:ERV:MER4D1
chr1    .       BED_feature     94751   95005   500     -       .       ID=BED_feature9;Name=39:ERV:MLT1H
chr1    .       BED_feature     99986   100626  500     -       .       ID=BED_feature10;Name=43:ERV:MamRep605

We then swap the name `BED_feature` for `genes`. The `-i` means to for `sed` operate in-place

In [ ]:
sed -i 's/BED_feature/erv/g' erv.gff3

We then swap the syntax one by one. First, we change `ID=` to `gene_id`, used by the GTF format.

In [ ]:
sed -i 's/ID=\([A-Za-z0-9_]\+\);/gene_id "\1"; /g' erv.gff3

We then do the same by changing `Name=` to `gene_name`.

In [ ]:
sed -i 's/Name=\([A-Za-z0-9\:_]\+\)/gene_name "\1"/g' erv.gff3

Lastly, we mark them as exons for STAR. This is a cheap hack to get STAR to recognise and map to these ERV sequences instead of ignoring them. 

In [ ]:
awk -F"\t" '$3=exon' erv.gff3

At the end, it should look something like this:

In [ ]:
chr1    .       exon    34597   34659   500     -       .       gene_id "erv1"; gene_name "12:ERV:MLT1L"
chr1    .       exon    34566   34972   500     -       .       gene_id "erv2"; gene_name "14:ERV:MLT1J2"
chr1    .       exon    40850   41007   500     -       .       gene_id "erv3"; gene_name "18:ERV:LTR16C"
chr1    .       exon    41804   42717   500     -       .       gene_id "erv4"; gene_name "19:ERV:ERV3-16A3_I"
chr1    .       exon    46766   46842   500     -       .       gene_id "erv5"; gene_name "20:ERV:LTR12B"
chr1    .       exon    70764   70893   500     -       .       gene_id "erv6"; gene_name "29:ERV:LTR89"
chr1    .       exon    91610   91678   500     -       .       gene_id "erv7"; gene_name "36:ERV:MER4E1"
chr1    .       exon    92187   92477   500     -       .       gene_id "erv8"; gene_name "37:ERV:MER4D1"
chr1    .       exon    94751   95005   500     -       .       gene_id "erv9"; gene_name "39:ERV:MLT1H"
chr1    .       exon    99986   100626  500     -       .       gene_id "erv10"; gene_name "43:ERV:MamRep605"

This is the command that appends this file to a combined gtf file, which presumably contains the gencode annotations and other stuff. Enjoy :)

In [ ]:
cat erv.gtf >> combined.gtf